# Movie rewievs

In [1]:
import pandas as pd
import numpy as np


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding

Using TensorFlow backend.


## data exploration

In [6]:
data_neg = pd.read_csv("./movie_reviews/movie_reviews/train/neg_rating.csv")

In [7]:
data_pos = pd.read_csv("./movie_reviews/movie_reviews/train/pos_rating.csv")

In [55]:
data_pos

,id,rating
0,0,9
1,1,8
2,2,9
3,3,7
4,4,7
5,5,9
6,6,10
7,7,7
8,8,10
9,9,7


Load the GloVe corpus

In [12]:
embeddings_index = {}
f = open('./glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] 
    coefs = np.asarray(values[1:], dtype='float32') 
    embeddings_index[word] = coefs
f.close()


In [29]:
def data_loader(data, sentiment):
    train = []

    for i  in data["id"]:
        file = open("./movie_reviews/movie_reviews/train/{0}/{1}.txt".format(sentiment,i), "r")
        train.append(file.readlines())
    return train

In [30]:
d = data_loader(data_neg, "neg")

In [26]:
d

[["I like British humor, I believe it's one of the best in the world. I like almost every British sitcom (okay... maybe not Monthy Python, some of the jokes were great, but some of them I didn't understand.), but this League of Gentlemen is just something good to make you sick. This show was good in some way; it helped me lost some weight because watching this piece of garbage make me feel I'm not hungry anymore. This is really just disgusting, sick and not even funny TV show and I wonder who is actually laughing at this stuff. I watched it for about 10 minutes and turned it off. It was so disgusting, watching men dressed in the woman with yellow teeth and urinating on the car... I mean... what's so funny about that??? It makes me wanna puke. No humor, just disturbing images and cheap, toilet laughs... I don't know... if you like this stuff... you go ahead... watch it... but to be honest, people watching and enjoying this must have some emotional problems. Garbage."],
 ["I knew that I 

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ira/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
import re
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
def removing_stop_words(data):
    lines_without_stopwords=[] 
    for seq in data: 
        seq = seq[0].lower()
        seq_by_words = re.findall(r'(?:\w+)', seq, flags = re.UNICODE) 
        new_line=[]
        for word in seq_by_words:
            if word not in stop:
                new_line.append(word)
        lines_without_stopwords.append(new_line)

    return lines_without_stopwords

In [44]:
d_neg = removing_stop_words(d)

In [52]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
MAX_NUM_WORDS = 4000
MAX_SEQUENCE_LENGTH = 400
def prep_with_tokenize_pad(texts):
   
    tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

   
    
    print("data shape: ", data.shape)

    return word_index, data

In [50]:
word_index_neg, data_neg = prep_with_tokenize_pad(d_neg)

Found 54871 unique tokens.
data shape:  (62500, 400)


In [56]:
neg_labels = [0 for i in range(data_neg.shape[0])]

In [57]:
from keras.initializers import Constant
def embed(word_index, data):
    EMBEDDING_DIM = embeddings_index.get('a').shape[0]
    num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if i > MAX_NUM_WORDS:
            continue
        embedding_vector = embeddings_index.get(word) 
        if embedding_vector is not None:

            embedding_matrix[i] = embedding_vector
    return embedding_matrix, num_words

In [58]:
embedding_matrix, num_words = embed(word_index_neg, data_neg)

## binary classification

In [59]:

model = Sequential()
model.add(Embedding(num_words, 300, input_length=400, weights= [embedding_matrix], trainable=False))

model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [62]:
model.fit(data_neg, np.array(neg_labels), validation_split=0.1, epochs = 1)

ValueError: Error when checking target: expected dense_1 to have shape (2,) but got array with shape (1,)

# rating prediction
